### Installation

In [ ]:
!pip install -q openai==1.55.3 tiktoken==0.6.0 datasets session-info --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cubla

### Imports

In [ ]:
# Import all Python packages required to access the Azure Open AI API.
# Import additional packages required to access datasets and create examples.

from openai import AzureOpenAI
import json
import random
import tiktoken
import session_info

import pandas as pd
import numpy as np

from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tabulate import tabulate

In [ ]:
#session_info.show()

### Authentication

**Writing/Creating the config.json file**

In [ ]:
# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "Key_placeholder",            #Replace it with your credentials
    "AZURE_OPENAI_ENDPOINT": "End_point_placeholder",      #Replace it with your credentials
    "AZURE_OPENAI_APIVERSION": "2024-05-01-preview", #Replace it with your credentials
    "CHATGPT_MODEL": "gpt-4o-mini"             #Replace it with your credentials
}

In [ ]:
# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

Config file created successfully!


In [ ]:
with open('config.json', 'r') as az_creds:
    data = az_creds.read()

In [ ]:
creds = json.loads(data)

In [ ]:
client = AzureOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version=creds["AZURE_OPENAI_APIVERSION"]
)

In [ ]:
chat_model_id = creds["CHATGPT_MODEL"]

### Utilities

In [ ]:
def num_tokens_from_messages(messages):

    """
    Return the number of tokens used by a list of messages.
    Adapted from the Open AI cookbook token counter
    """

    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    # Each message is sandwiched with <|start|>role and <|end|>
    # Hence, messages look like: <|start|>system or user or assistant{message}<|end|>

    tokens_per_message = 3 # token1:<|start|>, token2:system(or user or assistant), token3:<|end|>

    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))

    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

    return num_tokens

In [ ]:
 cs_reviews_df = pd.read_csv('/content/courier-service_reviews-1.csv')
# Read CSV File Here

In [ ]:
cs_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         131 non-null    int64 
 1   review     131 non-null    object
 2   sentiment  131 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.2+ KB


In [ ]:
cs_reviews_df.sample(5)

,id,review,sentiment
49,50,ExpressWay Logistics' delivery drivers have re...,Negative
14,15,I recently switched to ExpressWay Logistics fo...,Positive
127,128,While ExpressWay Logistics' pricing is competi...,Negative
76,77,I had a terrible experience with ExpressWay Lo...,Negative
66,67,ExpressWay Logistics offers a wide range of sh...,Negative


In [ ]:
cs_reviews_df['sentiment'].value_counts()

,count
sentiment,
Positive,68
Negative,63


In [ ]:
cs_reviews_df.shape

(131, 3)

In [ ]:
cs_examples_df, cs_gold_examples_df = train_test_split(
    cs_reviews_df, #<- the full dataset
    test_size = 0.2, #<- 20% random sample selected for gold examples
    random_state=42 #<- ensures that the splits are the same for every session
)

In [ ]:
(cs_examples_df.shape, cs_gold_examples_df.shape)

((104, 3), (27, 3))

In [ ]:
columns_to_select = ['review','sentiment']

In [ ]:
gold_examples = (
        cs_gold_examples_df.loc[:, columns_to_select]
                                     .sample(21, random_state=42) #<- ensures that gold examples are the same for every session
                                     .to_json(orient='records')
)

In [ ]:
gold_examples

'[{"review":"The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive\'s efforts. The packaging seemed more than adequate to protect the contents during transit.","sentiment":"Positive"},{"review":"ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.","sentiment":"Negative"},{"review":"ExpressWay Logistics\' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their fa

In [ ]:
json.loads(gold_examples)[0]     #Json format

{'review': "The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.",
 'sentiment': 'Positive'}

##**Derive Prompt**

In [ ]:
user_message_template = """```{courier_service_review}```"""

courier_service_review = "The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care. Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit."

**Write Zero Shot System Message**

In [ ]:
zero_shot_system_message = """
You are an assistant that analyzes courier service reviews and identifies the sentiment expressed in the review.
The sentiment can either be 'positive', 'negative', or 'neutral'. Provide th sentiment along with a brief explination for you analysis.
"""
# Write zero shot system message here


**Create Zero Shot Prompt**

In [ ]:
zero_shot_prompt = f"""Analyze the following courier service review and determine its sentiment: {courier_service_review}
Provide the sentiment ('positive', 'negative', or 'neutral') and a brief explanation for your classification."""
# Create zero shot prompt to be input ready for completion function



In [ ]:
messages = [
    {"role": "system", "content": zero_shot_system_message},
    {"role": "user", "content": zero_shot_prompt},
]
num_tokens_from_messages(messages)

148

**Write Few Shot System Message**

In [ ]:
few_shot_system_message = """You are an assistant skilled at determining the sentiment of courier service reviews.
Based on examples provided, you will classify the sentiment of a given review as 'positive', 'negative', or 'neutral'.
Additionally, provide a brief explination for your sentiment classification."""

In [ ]:
def create_examples(dataset, n=4):

    """
    Return a JSON list of randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (review + label)
        n (int): number of examples of each class to be selected

    Output:
        randomized_examples (JSON): A JSON with examples in random order
    """

    positive_reviews = (dataset.sentiment == 'Positive')
    negative_reviews = (dataset.sentiment == 'Negative')
    columns_to_select = ['review', 'sentiment']

    positive_examples = dataset.loc[positive_reviews, columns_to_select].sample(n)
    negative_examples = dataset.loc[negative_reviews, columns_to_select].sample(n)

    examples = pd.concat([positive_examples, negative_examples])

    # sampling without replacement is equivalent to random shuffling

    randomized_examples = examples.sample(2*n, replace=False)

    return randomized_examples.to_json(orient='records')

**Create Examples For Few shot prompts**

In [ ]:
examples = '''
[
    {"review": "Expressway Logistics is the worst courier service I've ever encountered. They consistently fail to meet delivery deadlines, leaving me scrambling to find alternative solutions. Their tracking system is a joke, providing inaccurate information that only adds to the frustration. If you enjoy stress and disappointment, by all means, use their services. Otherwise, steer clear.", "sentiment": "Negative"},
    {"review": "Expressway Logistics' comprehensive tracking system provides real-time updates on the status of our shipments, giving us peace of mind throughout the shipping process. Their advanced tracking capabilities allow us to monitor shipments closely and proactively address any issues that may arise. With Expressway Logistics, we can trust that our packages are in good hands.", "sentiment": "Positive"},
    {"review": "I am extremely disappointed with the service provided by Expressway Logistics. My parcel was delivered late and damaged, and the customer support team was unhelpful in resolving the issue and closed my support ticket without any satisfactory resolution. This was not expected!", "sentiment": "Negative"},
    {"review": "As a frequent traveler, I often rely on shipping services to send luggage and personal belongings ahead to my destination. However, I've had mixed experiences with other logistics providers in the past, which made me hesitant to try new ones. That changed when I decided to give Expressway Logistics a try for my latest trip. From the moment I contacted them, I was impressed by their professionalism and attention to detail. I am impressed by their efforts and service.", "sentiment": "Positive"}
]
'''
# Create Examples

In [ ]:
json.loads(examples)

[{'review': "Expressway Logistics is the worst courier service I've ever encountered. They consistently fail to meet delivery deadlines, leaving me scrambling to find alternative solutions. Their tracking system is a joke, providing inaccurate information that only adds to the frustration. If you enjoy stress and disappointment, by all means, use their services. Otherwise, steer clear.",
  'sentiment': 'Negative'},
 {'review': "Expressway Logistics' comprehensive tracking system provides real-time updates on the status of our shipments, giving us peace of mind throughout the shipping process. Their advanced tracking capabilities allow us to monitor shipments closely and proactively address any issues that may arise. With Expressway Logistics, we can trust that our packages are in good hands.",
  'sentiment': 'Positive'},
 {'review': 'I am extremely disappointed with the service provided by Expressway Logistics. My parcel was delivered late and damaged, and the customer support team was

In [ ]:
def create_prompt(system_message, examples, user_message_template):

    """
    Return a prompt message in the format expected by the Open AI API.
    Loop through the examples and parse them as user message and assistant
    message.

    Args:
        system_message (str): system message with instructions for sentiment analysis
        examples (str): JSON string with list of examples
        user_message_template (str): string with a placeholder for courier service reviews

    Output:
        few_shot_prompt (List): A list of dictionaries in the Open AI prompt format
    """

    few_shot_prompt = [{'role':'system', 'content': system_message}]

    for example in json.loads(examples):
        example_review = example['review']
        example_sentiment = example['sentiment']

        few_shot_prompt.append(
            {
                'role': 'user',
                'content': user_message_template.format(
                    courier_service_review=example_review
                )
            }
        )

        few_shot_prompt.append(
            {'role': 'assistant', 'content': f"{example_sentiment}"}
        )

    return few_shot_prompt

**Create Few Shot Prompt**

In [ ]:
few_shot_prompt = [
    {'role': 'system',
     'content': "You are a customer review sentiment classifier assistant. You will be provided with a customer review. Each review will be enclosed by triple backticks. You need to read the review and then classify it into one of the below two types:\n"
                "a) 'Positive'\n"
                "b) 'Negative'\n"
                "Classify the review in the above 2 sentiments only. DO NOT use any other sentiment. Sentiments like Neutral and Mixed are Unacceptable. If the review contains both positive and negative sentiment, still classify into one of them and this will be our FINAL SENTIMENT. Now this FINAL SENTIMENT should be the output in these 2 sentiments - 'Positive' and 'Negative' are only acceptable answers.\n"
                "Compulsory - Answer should only contain one word mentioning FINAL SENTIMENT - either 'Positive' or 'Negative'. Do NOT explain your answer.\n"
                "Repeat, unacceptable responses - Neutral and Mixed. Every review is compulsory to be only a 'Positive' or 'Negative'\n"},
    {'role': 'user',
     'content': "```Expressway Logistics is the worst courier service I've ever encountered. They consistently fail to meet delivery deadlines, leaving me scrambling to find alternative solutions. Their tracking system is a joke, providing inaccurate information that only adds to the frustration. If you enjoy stress and disappointment, by all means, use their services. Otherwise, steer clear.```"},
    {'role': 'assistant',
     'content': 'Negative'},
    {'role': 'user',
     'content': "```Expressway Logistics' comprehensive tracking system provides real-time updates on the status of our shipments, giving us peace of mind throughout the shipping process. Their advanced tracking capabilities allow us to monitor shipments closely and proactively address any issues that may arise. With Expressway Logistics, we can trust that our packages are in good hands.```"},
    {'role': 'assistant',
     'content': 'Positive'},
    {'role': 'user',
     'content': "```I am extremely disappointed with the service provided by Expressway Logistics. My parcel was delivered late and damaged, and the customer support team was unhelpful in resolving the issue and closed my support ticket without any satisfactory resolution. This was not expected!```"},
    {'role': 'assistant',
     'content': 'Negative'},
    {'role': 'user',
     'content': "```As a frequent traveler, I often rely on shipping services to send luggage and personal belongings ahead to my destination. However, I've had mixed experiences with other logistics providers in the past, which made me hesitant to try new ones. That changed when I decided to give Expressway Logistics a try for my latest trip. From the moment I contacted them, I was impressed by their professionalism and attention to detail. I am impressed by their efforts and service.```"},
    {'role': 'assistant',
     'content': 'Positive'}
]
# Create Few shot prompt

In [ ]:
few_shot_prompt

[{'role': 'system',
  'content': "You are a customer review sentiment classifier assistant. You will be provided with a customer review. Each review will be enclosed by triple backticks. You need to read the review and then classify it into one of the below two types:\na) 'Positive'\nb) 'Negative'\nClassify the review in the above 2 sentiments only. DO NOT use any other sentiment. Sentiments like Neutral and Mixed are Unacceptable. If the review contains both positive and negative sentiment, still classify into one of them and this will be our FINAL SENTIMENT. Now this FINAL SENTIMENT should be the output in these 2 sentiments - 'Positive' and 'Negative' are only acceptable answers.\nCompulsory - Answer should only contain one word mentioning FINAL SENTIMENT - either 'Positive' or 'Negative'. Do NOT explain your answer.\nRepeat, unacceptable responses - Neutral and Mixed. Every review is compulsory to be only a 'Positive' or 'Negative'\n"},
 {'role': 'user',
  'content': "```Expressway

In [ ]:
num_tokens_from_messages(few_shot_prompt)

510

##**Evaluate prompts**

In [ ]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for courier service review

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths, review_texts = [], [], []

    for example in json.loads(gold_examples):
        gold_input = example['review']
        user_input = [
            {
                'role':'user',
                'content': user_message_template.format(courier_service_review=gold_input)
            }
        ]

        try:
            response = client.chat.completions.create(
                model=chat_model_id,
                messages=prompt+user_input,
                temperature=0, # <- Note the low temperature (For a deterministic response)
                max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            )

            prediction = response.choices[0].message.content
            # response = openai.ChatCompletion.create(
            #     deployment_id=chat_model_id,
            #     messages=prompt+user_input,
            #     temperature=0, # <- Note the low temperature(For a deterministic response)
            #     # max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            # )

            # prediction = response['choices'][0]['message']['content']
            model_predictions.append(prediction.strip()) # <- removes extraneous white spaces
            ground_truths.append(example['sentiment'])
            review_texts.append(gold_input)

        except Exception as e:
            continue
    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    table_data = [[text, pred, truth] for text, pred, truth in zip(review_texts, model_predictions, ground_truths)]
    headers = ["Review", "Model Prediction", "Ground Truth"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

    return micro_f1_score


**Evaluate zero shot prompt**

In [ ]:
# Initialize zero-shot performance list
zero_shot_performance = []

# Define the zero-shot prompt
zero_shot_prompt = [{'role': 'system', 'content': zero_shot_system_message}]

# Evaluate the zero-shot prompt
zero_shot_micro_f1 = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

# Append the result to the zero-shot performance list
zero_shot_performance.append(zero_shot_micro_f1)

# Print the results
print(f"Zero-Shot Micro-F1 Score: {zero_shot_micro_f1}")

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

**Evaluate few shot prompt**

In [ ]:
# Initialize few-shot performance list
few_shot_performance = []

# Assemble the few-shot prompt
few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)

# Evaluate the few-shot prompt
few_shot_micro_f1 = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

# Append the result to the few-shot performance list
few_shot_performance.append(few_shot_micro_f1)

# Print the results
print(f"Few-Shot Micro-F1 Score: {few_shot_micro_f1}")

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
num_eval_runs = 5

In [ ]:
zero_shot_performance = []
few_shot_performance = []

In [ ]:
for _ in tqdm(range(num_eval_runs)):

    # For each run create a new sample of examples
    examples = create_examples(cs_examples_df)

    # Assemble the zero shot prompt with these examples
    zero_shot_prompt = [{'role':'system', 'content': zero_shot_system_message}]
    # zero_shot_prompt = create_prompt(zero_shot_system_message, examples, user_message_template)

    # Assemble the few shot prompt with these examples
    few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)

    # Evaluate zero shot prompt accuracy on gold examples
    zero_shot_micro_f1 = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

    # Evaluate few shot prompt accuracy on gold examples
    few_shot_micro_f1 = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

    zero_shot_performance.append(zero_shot_micro_f1)
    few_shot_performance.append(few_shot_micro_f1)

  0%|          | 0/5 [00:00<?, ?it/s]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 20%|██        | 1/5 [02:02<08:11, 122.79s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 40%|████      | 2/5 [04:09<06:15, 125.21s/it]

...

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 80%|████████  | 4/5 [08:53<02:18, 138.23s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

100%|██████████| 5/5 [10:54<00:00, 130.85s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

**Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt**

In [ ]:
zero_shot_mean = np.mean(zero_shot_performance)
zero_shot_std = np.std(zero_shot_performance)
print(f"Zero-Shot: Mean = {zero_shot_mean:.2f}, Std = {zero_shot_std:.2f}")
# Calculate for Zero Shot

Zero-Shot: Mean = 0.00, Std = 0.00


In [ ]:
few_shot_mean = np.mean(few_shot_performance)
few_shot_std = np.std(few_shot_performance)
print(f"Few-Shot: Mean = {few_shot_mean:.2f}, Std = {few_shot_std:.2f}")
# Calculate for Few Shot

Few-Shot: Mean = 0.81, Std = 0.00


##**Observation and Insights and Business perspective**

Positive reviews: 51%
Negative reviews: 48%

Positive reviews:
- Customers appreciate the professionalism and reliability of ExpressWay
  Logistics in timely deliveries and robust tracking mechanisms.
- Positive reviews emphasize the value of customer service and
  competitive pricing.

Negative reviews
- Common issues include delayed deliveries, damaged packages, and
  inadequate responses from customer support.
- Complaints often focus on the lack of accountability for fragile items
  and issues with grievances.

Positive feedback can be leveraged to enchance marketing efforts and boost credibility.
Negative feedback can be used to better address customer complaints and implement a system to handle these issues in a more timely manner.

Sentiment trends can be used to better develope delivery routes, employee training, protocals and other various aspects of the business.